# Check out specific fungal taxa in the context of some metadata

In [3]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries

In [1]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries


In [2]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.feature_table.methods import (filter_samples, filter_seqs)
from sklearn.metrics import r2_score
import numpy as np
import re

from qiime2.plugins import diversity as q2d

%matplotlib inline

# Load data

--> Fungi OTU clustered and already annotated! 

In [3]:
df_ITS = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berries-closedRefOTU/berries_harvest_cOTUs_rarefied_labled.tsv', sep='\t', index_col=0)
df_ITS.index.name = "SAMPLE_NAME"

df_ITS

,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Schizophyllaceae;g__Schizophyllum;s__Schizophyllum_commune,k__Fungi;p__Ascomycota;c__unidentified;o__unidentified;f__unidentified;g__unidentified;s__unidentified,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Crepidotaceae;g__Crepidotus;s__Crepidotus_stenocystis,k__Fungi;p__Basidiomycota;c__Tremellomycetes;o__Filobasidiales;f__Filobasidiaceae;g__Filobasidium;s__Filobasidium_magnum,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Sporidiobolales;f__Sporidiobolaceae;g__Sporobolomyces;s__Sporobolomyces_roseus,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Microbotryomycetes_ord_Incertae_sedis;f__Microbotryomycetes_fam_Incertae_sedis;g__Curvibasidium;s__Curvibasidium_cygneicollum,k__Fungi;p__Basidiomycota;c__Tremellomycetes;o__Tremellales;f__Bulleribasidiaceae;g__Derxomyces;s__Derxomyces_mrakii,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Phaeosphaeriaceae;g__Leptospora;s__unidentified,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Russulales;f__Bondarzewiaceae;g__Heterobasidion;s__Heterobasidion_parviporum,k__Fungi;p__Ascomycota;c__unidentified;o__unidentified;f__unidentified;g__unidentified;s__unidentified.1,...,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Xylariales;f__Xylariaceae;g__Barrmaelia;s__Barrmaelia_rappazii,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Tricholomataceae;g__Macrocystidia;s__Macrocystidia_cucumis,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Tricholomataceae;g__Macrocystidia;s__Macrocystidia_cucumis.1,k__Fungi;p__Ascomycota;c__Pezizomycetes;o__Pezizales;f__Pezizaceae;g__Iodophanus;s__Iodophanus_testaceus,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__unidentified;g__unidentified;s__unidentified.28,k__Fungi;p__Ascomycota;c__Pezizomycetes;o__Pezizales;f__Pyronemataceae;g__Wilcoxina;s__unidentified,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__unidentified.3,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Mycosphaerellaceae;g__Passalora;s__Passalora_passaloroides,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Pterulaceae;g__Pterula;s__Pterula_gracilis,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Auriculariales;f__unidentified;g__unidentified;s__unidentified.1
SAMPLE_NAME,,,,,,,,,,,,,,,,,,,,,
Lavaux_2021-09-30_must_Plot4_Row9/Vine20_ID6,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot15_Row3/Vine20_ID121b,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot17_Row13/Vine20_ID171,0.0,0.0,0.0,0.0,3.0,227.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot5_Row6/Vine30_ID37b,0.0,0.0,0.0,209.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot11_Row6/Vine22_ID66,0.0,0.0,0.0,64.0,0.0,106.0,0.0,3.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lavaux_2021-09-30_must_Plot18_Row2/Vine7_ID185,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot20_Row7/Vine20_ID207,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot9_Row11/Vine22_ID54,0.0,0.0,0.0,103.0,0.0,12.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Plant Pathogens

In [26]:
# UNITE fungal list
unite_df = pd.read_csv('/home/lfloerl/microterroir/Microbiome/ITS/UNITE-FungalTraits.csv')

# Step 1: Filter unite_df for rows where primary_lifestyle is 'plant_pathogen'
filtered_unite_df = unite_df[unite_df['primary_lifestyle'].str.contains('plant_pathogen', na=False)]
# Step 2: Extract GENUS from the filtered unite_df
genus_list = filtered_unite_df['GENUS'].tolist()

len(genus_list)

13961

#### let's check if we need to use the Macrofungi FeatureTable 

> -> no, most pathogens (87.62%) are not macrofungi and none of the important ones are 

In [ ]:
# Filter unite_df for rows where primary_lifestyle is 'plant_pathogen'
filtered_unite_df = unite_df[unite_df['primary_lifestyle'].str.contains('plant_pathogen', na=False)]

# List of fruiting body types associated with macrofungi
fruting_body_forming = [
    'agaricoid', 'apothecium_(hymenium on surface)', 'clathroid', 'clavarioid', 
    'corticioid', 'cyphelloid', 'gasteroid', 'gasteroid-hypogeous', 
    'hysterothecium_(opening_slith-like)', 'mazaedium_(pushpin-like)', 
    'phalloid', 'polyporoid', 'tremelloid'
]

# Extract Taxon names of macrofungi
macrofungi_taxa = unite_df[unite_df['Fruitbody_type_template'].isin(fruting_body_forming)]['Taxon'].tolist()

# Count total plant pathogens
total_plant_pathogens = len(filtered_unite_df)

# Count plant pathogens that are NOT macrofungi (i.e., microfungi)
microfungi_count = sum(~filtered_unite_df['Taxon'].isin(macrofungi_taxa))

# Calculate percentage
percentage_microfungi = (microfungi_count / total_plant_pathogens) * 100 if total_plant_pathogens > 0 else 0

print(f"Percentage of plant pathogens that are microfungi: {percentage_microfungi:.2f}%")

Percentage of plant pathogens that are microfungi: 87.62%


In [20]:
# Filter plant pathogens that are macrofungi
macrofungi_pathogens = filtered_unite_df[filtered_unite_df['Taxon'].isin(macrofungi_taxa)]

# Extract the genus (assuming "Genus species" format)
macrofungi_pathogens['Genus'] = macrofungi_pathogens['Taxon'].str.split().str[0]

# Get unique genera
unique_macrofungi_genera = macrofungi_pathogens['Genus'].unique()

# Print results
print("Genera of plant pathogens that are macrofungi:")
print(unique_macrofungi_genera)

Genera of plant pathogens that are macrofungi:
['k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Cantharellales;f__Ceratobasidiaceae;g__Ceratobasidium;s__Ceratobasidium_sp'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Corticiales;f__Corticiaceae;g__Laetisaria;s__Laetisaria_sp'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Typhulaceae;g__Typhula;s__Typhula_ishikariensis'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Agaricales_fam_Incertae_sedis;g__Rimbachia;s__Rimbachia_sp'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Typhulaceae;g__Typhula;s__Typhula_fistulosa'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Polyporales;f__Meripilaceae;g__Meripilus;s__Meripilus_giganteus'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Corticiales;f__Corticiaceae;g__Limonomyces;s__Limonomyces_culmigenus'
 'k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Corticiales;f__Corticiaceae;g__Laetisaria;s__Laetisaria_agaves'
 'k__Fungi;p__Basidiomycota;c_

/scratch/lfloerl/tmpdata/ipykernel_1420785/498714753.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macrofungi_pathogens['Genus'] = macrofungi_pathogens['Taxon'].str.split().str[0]


#### now actually filter

In [9]:
# Create a regex pattern to match any term in the list
pattern = "|".join(genus_list)
# Filter columns based on regex pattern matching the genus in the taxa names
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col)]
# Filter the dataframe to only include the columns that match the pattern
pathogen_df = df_ITS[columns_to_keep]
pathogen_df.shape

(317, 354)

In [11]:
pathogen_df.to_csv('FunPathogens_crOTU90_annotated.csv')

# Selected Fungal Pathogens (Grapevine relevant)

In [22]:
genus_list = ["Botrytis", "Phaeomoniella", "Phaeoacremonium","Neofusicoccum", "Ilyonectria",  "Lasiodiplodia", "Fomitiporia", "Diplodia", "Diaporthe", "Neoscytalidium", "Biscogniauxia", 
              "Botryosphaeria",  "Plasmopara", "Erysiphaceae","Neonectria"] 

pattern = "|".join(genus_list)
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col, re.IGNORECASE)]
GV_pathogen_df = df_ITS[columns_to_keep]

# Output the shape to confirm selection
GV_pathogen_df.shape

(317, 28)

In [23]:
GV_pathogen_df.to_csv('FunGrapevinePathogens_crOTU90_annotated.csv')

In [24]:
GV_pathogen_df.head()

,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Diaporthales;f__Diaporthaceae;g__Diaporthe;s__Diaporthe_cynaroidis,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Sclerotiniaceae;g__Botrytis;s__Botrytis_cinerea,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Macrophoma;s__unidentified,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Diaporthales;f__Diaporthaceae;g__Diaporthe;s__Diaporthe_lusitanicae,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Dothiorella;s__Dothiorella_iberica,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__unidentified;g__unidentified;s__unidentified,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Phaeobotryon;s__Phaeobotryon_rhois,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Diaporthales;f__Diaporthaceae;g__Diaporthe;s__Diaporthe_helicis,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Diplodia;s__Diplodia_scrobiculata,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Sphaeropsis;s__Sphaeropsis_sapinea,...,k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Phaeomoniellales;f__unidentified;g__unidentified;s__unidentified,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Erysiphales;f__Erysiphaceae;g__Blumeria;s__Blumeria_graminis,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__Botryosphaeriaceae;g__Dothiorella;s__Dothiorella_viticola,k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Phaeomoniellales;f__Phaeomoniellaceae;g__Moristroma;s__Moristroma_quercinum,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Xylariales;f__Xylariaceae;g__Biscogniauxia;s__Biscogniauxia_marginata,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Erysiphales;f__Erysiphaceae;g__Blumeria;s__Blumeria_graminis.1,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Erysiphales;f__Erysiphaceae;g__Golovinomyces;s__Golovinomyces_riedlianus,k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Phaeomoniellales;f__Phaeomoniellaceae;g__Aequabiliella;s__Aequabiliella_effusa,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Diaporthales;f__Diaporthaceae;g__Diaporthe;s__Diaporthe_passiflorae,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__unidentified;g__unidentified;s__unidentified.1
SAMPLE_NAME,,,,,,,,,,,,,,,,,,,,,
Lavaux_2021-09-30_must_Plot4_Row9/Vine20_ID6,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot15_Row3/Vine20_ID121b,0.0,3558.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot17_Row13/Vine20_ID171,0.0,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot5_Row6/Vine30_ID37b,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot11_Row6/Vine22_ID66,1.0,523.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Plant Pathogens - ASVs

In [ ]:
df_ITS = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berries_harvest_ASVs_rarefied_labled.tsv', sep='\t', index_col=0)
df_ITS.index.name = "SAMPLE_NAME"

# Create a regex pattern to match any term in the list
pattern = "|".join(genus_list)
# Filter columns based on regex pattern matching the genus in the taxa names
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col)]
# Filter the dataframe to only include the columns that match the pattern
pathogen_df = df_ITS[columns_to_keep]
pathogen_df.shape

pathogen_df.to_csv('FunPathogens_ASVs_annotated.csv')

,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1302074.10FU,k__Fungi;p__Basidiomycota;c__Tremellomycetes;o__Cystofilobasidiales;f__Cystofilobasidiaceae;g__Cystofilobasidium;s__Cystofilobasidium_macerans;sh__SH1083411.10FU,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Torulaceae;g__Torula;s__Torula_fici;sh__SH1349958.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Didymellaceae;g__Neoascochyta;s__Neoascochyta_exitialis;sh__SH1213549.10FU,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1302074.10FU.1,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Boletales;f__Boletaceae;g__Boletus;s__Boletus_satanas;sh__SH1216224.10FU,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Sclerotiniaceae,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Sclerotiniaceae.1,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Lycoperdaceae;g__Lycoperdon;s__Lycoperdon_pyriforme;sh__SH1397992.10FU,...,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1302074.10FU.314,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Didymellaceae;g__Neoascochyta;s__Neoascochyta_tardicrescens;sh__SH1213171.10FU.2,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp.933,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Sclerotiniaceae.393,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Xylariales;f__Diatrypaceae;g__Cryptovalsa;s__Cryptovalsa_ampelina;sh__SH1052393.10FU.3,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Sporidiobolales;f__Sporidiobolaceae;g__Sporobolomyces;s__Sporobolomyces_patagonicus;sh__SH1273307.10FU.227,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1302074.10FU.315,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Glomerellales;f__Plectosphaerellaceae.16,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Microbotryomycetes_ord_Incertae_sedis;f__Microbotryomycetes_fam_Incertae_sedis;g__Curvibasidium;s__Curvibasidium_cygneicollum;sh__SH1189363.10FU.281,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1302074.10FU.316
SAMPLE_NAME,,,,,,,,,,,,,,,,,,,,,
364525_481-LP3-ITS-0481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364525_482-LP3-ITS-0482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364525_483-LP3-ITS-0483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364525_484-LP3-ITS-0484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364525_485-LP3-ITS-0485,0.0,0.0,0.0,0.0,0.0,68.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Fermentative Yeasts

In [14]:
terms = [
    "Saccharomyces", "Saccharomycodes", "Schizosaccharomyces",
    "Candida", "Torulaspora", "Debaryomyces", "Issatchenkia",
    "Pichia", "Kluyveromyces", "Metschnikowia", "Hanseniaspora",
    "Kloeckera", "Rhodotorula", "Brettanomyces", "Dekkera",
    "Zygosaccharomyces"]
# Create a regex pattern to match any term in the list
pattern = "|".join(terms)

# Filter columns based on regex pattern matching the genus in the taxa names
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col)]
# Filter the dataframe to only include the columns that match the pattern
fermentative_df = df_ITS[columns_to_keep]

fermentative_df.shape

(317, 8)

In [18]:
!pwd

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries


In [ ]:
fermentative_df.to_csv('FermentativeOrganisms_crOTU90_annotated.csv')